In [8]:
import pandas as pd
import numpy as np
import os
import requests
import json
from bs4 import BeautifulSoup

In [9]:
print(os.getcwd())

/Users/eugenechua/Downloads/AI_Builder_project


Why did I resort to using selenium? This is because when i tried using the basic request library, the HTML content output returned by **request.get** does not contain the job posting links, hence meaning they are most likely loaded dynamically by javascript.

Request cannot handle javascript! so I am only seeing current static content.

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from tqdm import tqdm
import time
import ast

In [5]:
# Set up Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode (no browser window)
options.add_argument("--disable-blink-features=AutomationControlled")  # Bypass bot detection
options.add_argument("--log-level=3")  # Suppress logs

# Launch browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open the MyCareersFuture page
url = "https://www.mycareersfuture.gov.sg/search?search=data%20scientist&sortBy=new_posting_date&page=0"
driver.get(url)

# Wait for JavaScript to load
time.sleep(5)  # Adjust if needed

job_link_list = []

# Extract job links using the correct class
job_elements = driver.find_elements(By.CLASS_NAME, "bg-white.mb3.w-100.dib.v-top.pa3.no-underline.flex-ns.flex-wrap.JobCard__card___22xP3")

print("Found Job Links:")
for job in job_elements:
    job_url = job.get_attribute("href")  # Extract job link
    if job_url:
        job_link_list.append(job_url)
        print(job_url)

# Close browser
driver.quit()

#Print the list of jobs found
print("\nTotal jobs Found:", len(job_link_list))

Found Job Links:
https://www.mycareersfuture.gov.sg/job/information-technology/manager-recruitpedia-77f621a9f6a42d6ad0c6145ef191bf6a?source=MCF&event=Search
https://www.mycareersfuture.gov.sg/job/information-technology/senior-data-engineer-singapore-exchange-eed0a4f0fa2d2ac7ab6705658b107971?source=MCF&event=Search
https://www.mycareersfuture.gov.sg/job/information-technology/data-scientist-8k-contract-evo-outsourcing-solutions-dc29fa8b88698f8693dbcafcacbde453?source=MCF&event=Search
https://www.mycareersfuture.gov.sg/job/information-technology/data-engineer-singapore-exchange-f32ca751004b50096f5e88dcab8d8c68?source=MCF&event=Search
https://www.mycareersfuture.gov.sg/job/healthcare/data-scientist-alexandra-hospital-national-university-health-system-f4112519a273476f578b096b49b0db60?source=MCF&event=Search
https://www.mycareersfuture.gov.sg/job/information-technology/data-engineer-idc-technologies-512bcb3ad8445a03e862f53a39c4aee4?source=MCF&event=Search
https://www.mycareersfuture.gov.sg/

In [ ]:
import csv

# Save job links to a CSV file
with open("data/job_links.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Job URL"])  # Header
    for link in job_link_list:
        writer.writerow([link])

print("Job links saved to job_links.csv!")


Job links saved to job_links.csv!


In [ ]:
job_postings_df = pd.read_csv('data/job_links.csv')
job_postings_df.head(10)

,Job URL
0,https://www.mycareersfuture.gov.sg/job/informa...
1,https://www.mycareersfuture.gov.sg/job/informa...
2,https://www.mycareersfuture.gov.sg/job/informa...
3,https://www.mycareersfuture.gov.sg/job/informa...
4,https://www.mycareersfuture.gov.sg/job/healthc...
5,https://www.mycareersfuture.gov.sg/job/informa...
6,https://www.mycareersfuture.gov.sg/job/informa...
7,https://www.mycareersfuture.gov.sg/job/consult...
8,https://www.mycareersfuture.gov.sg/job/informa...
9,https://www.mycareersfuture.gov.sg/job/informa...


In [8]:
job_link_list = [job_url for job_url in job_postings_df['Job URL']]
job_link_list

['https://www.mycareersfuture.gov.sg/job/information-technology/manager-recruitpedia-77f621a9f6a42d6ad0c6145ef191bf6a?source=MCF&event=Search',
 'https://www.mycareersfuture.gov.sg/job/information-technology/senior-data-engineer-singapore-exchange-eed0a4f0fa2d2ac7ab6705658b107971?source=MCF&event=Search',
 'https://www.mycareersfuture.gov.sg/job/information-technology/data-scientist-8k-contract-evo-outsourcing-solutions-dc29fa8b88698f8693dbcafcacbde453?source=MCF&event=Search',
 'https://www.mycareersfuture.gov.sg/job/information-technology/data-engineer-singapore-exchange-f32ca751004b50096f5e88dcab8d8c68?source=MCF&event=Search',
 'https://www.mycareersfuture.gov.sg/job/healthcare/data-scientist-alexandra-hospital-national-university-health-system-f4112519a273476f578b096b49b0db60?source=MCF&event=Search',
 'https://www.mycareersfuture.gov.sg/job/information-technology/data-engineer-idc-technologies-512bcb3ad8445a03e862f53a39c4aee4?source=MCF&event=Search',
 'https://www.mycareersfutur

In [9]:

# Set up Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode (no browser window)
options.add_argument("--disable-blink-features=AutomationControlled")  # Bypass bot detection
options.add_argument("--log-level=3")  # Suppress logs

# Launch browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Lists to store scraped data
job_titles_list = []
job_descriptions_requirements_list = []  # Merging both into one field


# Loop through job links
for job_url in tqdm(job_link_list, desc='Processing Job Listings', unit='job'):
    driver.get(job_url)
    time.sleep(5)  # Wait for JavaScript to load
    
    # 1️⃣ Extract Job Title
    jobtitle = driver.title.strip()  # Remove leading/trailing spaces
    job_titles_list.append(jobtitle)

    # 2️⃣ Extract All <li> Items from All <ul> Tags
    try:
        ul_elements = driver.find_elements(By.TAG_NAME, "ul")  # Find all <ul> elements
        all_li_items = []  # List to store all <li> text
        # all_li_items contains all the li items in a long list so loop through it now
        for ul in ul_elements:
            li_elements = ul.find_elements(By.TAG_NAME, "li")  # Find all <li> under each <ul>
            for li in li_elements:
                cleaned_text = li.text.strip()  # Clean up each <li> item
                if cleaned_text:  # Only add if it's not empty
                    all_li_items.append(f"- {cleaned_text}")

        # Combine all extracted <li> into a single text block
        job_description_requirements = "\n".join(all_li_items) if all_li_items else "Not Found"
    except:
        job_description_requirements = "Not Found"

    job_descriptions_requirements_list.append(job_description_requirements)

# Close browser
driver.quit()

# Create DataFrame
job_df = pd.DataFrame({
    "Job_Title": job_titles_list,
    "Job_Desc_Req": job_descriptions_requirements_list
})



Processing Job Listings: 100%|██████████| 20/20 [01:53<00:00,  5.69s/job]


In [22]:
# ✅ Remove all "-" from Job Descriptions column
job_df["Job_Desc_Req"] = job_df["Job_Desc_Req"].str.replace(r"^- ", "", regex=True).str.replace(r"\n- ", "\n", regex=True)


In [23]:
job_df['Job_Desc_Req'][0]

'Collaborate with data scientists to understand their data, processing, and infrastructure needs to support AI and machine learning initiatives effectively.\nEnsure IT infrastructure is capable of supporting AI applications by providing high-performance computing resources and cloud-based platforms.\nFacilitate data accessibility across various sources, such as CRM, ERP, and e-commerce systems, while ensuring security and compliance.\nWork with data scientists to operationalize machine learning models by setting up deployment, monitoring, and continuous improvement frameworks.\nEvaluate and implement AI/ML platforms, tools, and technologies to enhance data science workflows, including model training and deployment.\nOversee data security protocols to protect sensitive information and ensure compliance with data privacy regulations.\nSupport data visualization initiatives and provide technical guidance to data science and BI teams for large-scale data analysis.\nManage IT systems for op

In [49]:
job_df.head(5)

,Job_Title,Job_Desc_Req
0,IT Manager (Data Science) Information Technolo...,Collaborate with data scientists to understand...
1,Senior Data Engineer Information Technology Jo...,"Design and implement data architecture, pipeli..."
2,DATA SCIENTIST | UP TO $8K | CONTRACT Informat...,"3-8 years of experience in computer science, a..."
3,Data Engineer Information Technology Jobs Sing...,"Design and implement data architecture, pipeli..."
4,Data Scientist (1-year contract)(7708) - Alexa...,Bachelor’s degree (required) in fields such as...


In [ ]:
#Exporting the scrapped job postings as backup first
job_df.to_csv('data/job_data_scrapped.csv', index=False)

## Onto Skills & Knowledge extractions now

In [26]:
from transformers import pipeline

token_skill_classifier = pipeline(model="jjzha/jobbert_skill_extraction", aggregation_strategy="first")
token_knowledge_classifier = pipeline(model="jjzha/jobbert_knowledge_extraction", aggregation_strategy="first")

Device set to use mps:0
Device set to use mps:0


## Setting up the span extraction helper functions

In [50]:
def aggregate_span(results):
    new_results = []
    current_result = results[0]

    for result in results[1:]:
        if result["start"] == current_result["end"] + 1:
            current_result["word"] += " " + result["word"]
            current_result["end"] = result["end"]
        else:
            new_results.append(current_result)
            current_result = result

    new_results.append(current_result)

    return new_results

In [51]:
def ner_skill(text):
    output_skills = token_skill_classifier(text)
    for result in output_skills:
        if result.get("entity_group"):
            result['"entity'] = "Skill"
            del result["entity_group"]
            
    if len(output_skills) > 0:
        output_skills = aggregate_span(output_skills)
        
    return {"entities": output_skills}

def ner_knowledge(text):
    output_knowledge = token_knowledge_classifier(text)
    for result in output_knowledge:
        if result.get("entity_group"):
            result["entity"] = "Knowledge"
            del result["entity_group"]
            
    if len(output_knowledge) > 0:
        output_knowledge = aggregate_span(output_knowledge)
        
    return {"entities": output_knowledge}

In [85]:
import math
import re

def long_chunk(text, context_length=512):
    """Splits long text into chunks of at most `context_length` words."""
    
    words = re.findall(r'\w+', text)  # Tokenize words once
    total_words = len(words)
    repeats = math.ceil(total_words / context_length)  # Calculate how many chunks we need
    
    chunks = []
    tracker = 0  # Start index
    
    for i in range(repeats):
        b = (i + 1) * context_length  # Compute end index for chunk
        chunked_text = ' '.join(words[tracker:b])  # Join words into a chunk
        chunks.append(chunked_text)  # Store the chunk
        tracker = b  # Move tracker to next chunk
    
    return chunks  # Return all chunks as a list

In [ ]:
from tqdm import tqdm  # Progress tracking

# Lists to store extracted skills and knowledge
skill_list = []
knowledge_list = []

# Iterate through the 'Job_Desc_Req' column
for row in tqdm(job_df['Job_Desc_Req'], desc='Extracting Skills and Knowledge'):
    # Handle long texts by chunking using the correct tokenizer
    text_chunks = long_chunk(row, context_length=200)

    extracted_skills = []
    extracted_knowledge = []

    for chunk in text_chunks:
        # Extract skills using Mike Zhang's model
        skills_extracted = ner_skill(chunk)
        skills = [u['word'] for u in skills_extracted['entities']]
        if skills:
            extracted_skills.extend(skills)

        # Extract knowledge using Mike Zhang's model
        knowledge_extracted = ner_knowledge(chunk)
        knowledge = [u['word'] for u in knowledge_extracted['entities']]
        if knowledge:
            extracted_knowledge.extend(knowledge)

    # Store results as lists
    skill_list.append(extracted_skills)
    knowledge_list.append(extracted_knowledge)

# Convert lists to comma-separated strings
job_df["Skills"] = skill_list
job_df["Knowledge"] = knowledge_list

# Convert lists to JSON strings before saving to CSV - this is very DELIBERATE as it will properly convert back to python lists when reloaded!
job_df["Skills"] = job_df["Skills"].apply(json.dumps)  # Convert list to JSON string
job_df["Knowledge"] = job_df["Knowledge"].apply(json.dumps)


'''
#Appending extracted skills and knowledge as additional fields to the original jobs posting dataframe
job_df['Skills'] = skill_list
'''

Extracting Skills and Knowledge: 100%|██████████| 20/20 [00:02<00:00,  7.72it/s]


"\n#Appending extracted skills and knowledge as additional fields to the original jobs posting dataframe\njob_df['Skills'] = skill_list\n"

In [98]:
job_df.head(10)

,Job_Title,Job_Desc_Req,Skills,Knowledge
0,IT Manager (Data Science) Information Technolo...,Collaborate with data scientists to understand...,"[""Collaborate with data scientists"", ""support ...","[""data"", ""AI"", ""machine learning"", ""IT"", ""AI"",..."
1,Senior Data Engineer Information Technology Jo...,"Design and implement data architecture, pipeli...","[""Design and implement data architecture pipel...","[""data architecture pipelines"", ""ETL"", ""data p..."
2,DATA SCIENTIST | UP TO $8K | CONTRACT Informat...,"3-8 years of experience in computer science, a...","[""building machine learning models at scale"", ...","[""computer science applied mathematics"", ""quan..."
3,Data Engineer Information Technology Jobs Sing...,"Design and implement data architecture, pipeli...","[""Design and implement data architecture pipel...","[""data architecture pipelines"", ""ETL"", ""data p..."
4,Data Scientist (1-year contract)(7708) - Alexa...,Bachelor’s degree (required) in fields such as...,"[""analytical and critical thinking skills"", ""e...","[""Bachelor s degree"", ""Data Science Computer S..."
5,"Data Engineer (Scala, SQL, Hadoop) Information...","Design, build, and optimize scalable data pipe...","[""Design build and optimize scalable data pipe...","[""data pipelines"", ""Scala"", ""Hadoop"", ""ETL"", ""..."
6,"Data Engineer (Scala, SQL, Hadoop) Information...","Design, build, and optimize scalable data pipe...","[""Design build and optimize scalable data pipe...","[""data pipelines"", ""Scala"", ""Hadoop"", ""ETL"", ""..."
7,"Service Delivery Manager, AWS Data Analytics -...",Oversee the end-to-end delivery of IT and data...,"[""Oversee the end to end delivery of IT and da...","[""IT"", ""data analytics"", ""delivery"", ""data ana..."
8,Data Architect Information Technology Jobs Sin...,Design conceptual and logical data models and ...,"[""Design conceptual and logical data models an...","[""Data"", ""database structure principles"", ""dat..."
9,Associate AI Scientist / AI Scientist – Artifi...,Push the frontiers of what is possible in the ...,"[""Push the frontiers of what is possible"", ""cr...","[""artificial intelligence"", ""machine learning""..."


In [ ]:
job_df.to_csv('data/job_data_processed.csv', index=False)

In [13]:
job_df = pd.read_csv('data/job_data_processed.csv')

job_df.head(10)

,Job_Title,Job_Desc_Req,Skills,Knowledge
0,IT Manager (Data Science) Information Technolo...,Collaborate with data scientists to understand...,"[""Collaborate with data scientists"", ""support ...","[""data"", ""AI"", ""machine learning"", ""IT"", ""AI"",..."
1,Senior Data Engineer Information Technology Jo...,"Design and implement data architecture, pipeli...","[""Design and implement data architecture pipel...","[""data architecture pipelines"", ""ETL"", ""data p..."
2,DATA SCIENTIST | UP TO $8K | CONTRACT Informat...,"3-8 years of experience in computer science, a...","[""building machine learning models at scale"", ...","[""computer science applied mathematics"", ""quan..."
3,Data Engineer Information Technology Jobs Sing...,"Design and implement data architecture, pipeli...","[""Design and implement data architecture pipel...","[""data architecture pipelines"", ""ETL"", ""data p..."
4,Data Scientist (1-year contract)(7708) - Alexa...,Bachelor’s degree (required) in fields such as...,"[""analytical and critical thinking skills"", ""e...","[""Bachelor s degree"", ""Data Science Computer S..."
5,"Data Engineer (Scala, SQL, Hadoop) Information...","Design, build, and optimize scalable data pipe...","[""Design build and optimize scalable data pipe...","[""data pipelines"", ""Scala"", ""Hadoop"", ""ETL"", ""..."
6,"Data Engineer (Scala, SQL, Hadoop) Information...","Design, build, and optimize scalable data pipe...","[""Design build and optimize scalable data pipe...","[""data pipelines"", ""Scala"", ""Hadoop"", ""ETL"", ""..."
7,"Service Delivery Manager, AWS Data Analytics -...",Oversee the end-to-end delivery of IT and data...,"[""Oversee the end to end delivery of IT and da...","[""IT"", ""data analytics"", ""delivery"", ""data ana..."
8,Data Architect Information Technology Jobs Sin...,Design conceptual and logical data models and ...,"[""Design conceptual and logical data models an...","[""Data"", ""database structure principles"", ""dat..."
9,Associate AI Scientist / AI Scientist – Artifi...,Push the frontiers of what is possible in the ...,"[""Push the frontiers of what is possible"", ""cr...","[""artificial intelligence"", ""machine learning""..."


In [14]:
#Converting Skills and Knowledge fields back to proper python lists
job_df['Skills'] = job_df['Skills'].apply(ast.literal_eval)
job_df['Knowledge'] = job_df['Knowledge'].apply(ast.literal_eval)

In [15]:
#Checking for Skills
print(job_df['Skills'][0])
print(type(job_df['Skills'][0]))

#Checking for Knowledge
print(job_df['Knowledge'])
print(type(job_df['Knowledge'][0]))
      

['Collaborate with data scientists', 'support AI and machine learning initiatives effectively Ensure IT infrastructure is capable', 'supporting AI applications', 'providing high performance computing resources and cloud based platforms Facilitate data accessibility', 'ensuring security and compliance Work with data scientists', 'operationalize machine learning models', 'setting up deployment monitoring and continuous improvement frameworks Evaluate and implement AI ML platforms tools and technologies', 'enhance data science workflows', 'Oversee data security protocols', 'protect sensitive information', 'ensure compliance with data privacy regulations Support data visualization initiatives', 'provide technical guidance to data science and BI teams', 'scale data analysis Manage IT systems for optimal performance ensuring scalability', 'data volumes grow and applications', 'overseeing model deployment processes', 'support data storytelling initiatives']
<class 'list'>
0     [data, AI, mac